## 初期設定

In [1]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
import src.config as cfg
import src.data_loader as dl
import src.data_cleaning_utils as cu
print("🔁 autoreload 有効化完了")
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import defaultdict
import hashlib

🔁 autoreload 有効化完了


In [2]:
# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

{'project_name': 'stock_screening',
 'data_path': {'raw': './data/raw',
  'interim': './data/interim',
  'processed': './data/processed',
  'reference': './data/reference'},
 'files': {'raw': ['fy-balance-sheet.csv',
   'fy-cash-flow-statement.csv',
   'fy-profit-and-loss.csv',
   'fy-stock-dividend.csv'],
  'reference': 'CodeData.csv',
  'interim': None,
  'processed': None},
 'years': [2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2020,
  2021,
  2022,
  2023,
  2024,
  2025],
 'na_values': [''],
 'output': {'base_path': './output'}}

## 本ファイルの説明
データが最低限の品質基準を満たしているかを網羅的に確認し、ファイルロードからクリーニングまでのプロセスを決定します。

<details>
<summary><b>結果</b></summary>


</details>


### データのロードの確認
ファイル重複やファイル欠損、破壊がないかを確かめ、プロセスを確定します。
<details>
<summary><b>結果</b></summary>

ファイルの重複や欠損はなく、正しくロードされました。  
修正処理は不要です。
</details>

In [ ]:
print("")
print("ファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"]
)
display(df_DATAs_BY_ALL_FILEs)

print("")
print("次に欠けてるファイルがないか確認します")
def chk_file_missing(df_DATAs_BY_ALL_FILEs):
    flg = True
    for (filename,year), df in df_DATAs_BY_ALL_FILEs.items():
        if df.empty:
            print("❌ ".filename,year,"データフレームが空です")
            flg = False
    if flg:
        print("✅ ファイル欠損はありません。")
chk_file_missing(df_DATAs_BY_ALL_FILEs)

print("")
print("重複ファイルがないか確認します")
def df_hash(df, ignore_order=False):
    data_only = df.copy().fillna("").astype(str)
    if ignore_order:
        # 行順も無視（全列ソートしてから比較）
        data_only = data_only.sort_values(by=data_only.columns.tolist()).reset_index(drop=True)
    # 列名を無視して内容だけでハッシュを生成
    h = hashlib.md5(pd.util.hash_pandas_object(data_only, index=True).values)
    return h.hexdigest()
hash_map = defaultdict(list)
for (fname, year), df in df_DATAs_BY_ALL_FILEs.items():
    h = df_hash(df, ignore_order=False)  # ←行順を無視したいなら True に
    hash_map[h].append((fname, year))
duplicates = {h: keys for h, keys in hash_map.items() if len(keys) > 1}
if duplicates:
    print("🔍 同じ内容のデータフレーム（ヘッダー無視）が見つかりました：\n")
    for h, key_group in duplicates.items():
        print(f"ハッシュ: {h}")
        for fname, year in key_group:
            print(f"  - {fname}（{year}）")
        print()
else:
    print("✅ 重複データフレームはありません。")

print("")
print("データロードの確認を終了します。")


### 欠損値表現の確認と処理
欠損値の表現を確認し、修正プロセスを決定します。

<details>
<summary><b>結果</b></summary>

"-"が欠損値として使われています。  
これはsetting.yamlのnan_valuesに登録し欠損値として読み込みます。  
  
また、ゼロも使われています。  
ゼロは値として残し、必要に応じてfeatures_generatorで再計算し更新します。

以下がゼロが使われている項目です。  
{'fy-balance-sheet.csv': ['BPS', '短期借入金', '総資産', '自己資本比率', '長期借入金']  
'fy-cash-flow-statement.csv': ['営業CF', '営業CFマージン', '投資CF', '現金同等物', '設備投資', '財務CF']  
'fy-profit-and-loss.csv': ['EPS', 'ROA', 'ROE', '営業利益', '売上高', '純利益', '経常利益'],  
'fy-stock-dividend.csv': ['一株配当', '剰余金の配当', '純資産配当率', '総還元性向', '自社株買い', '配当性向']}
</details>

In [ ]:
# 欠損値の表現確認
print("")
print("ファイル毎にデータを取得します")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"]
)
#display(df_DATAs_BY_ALL_FILEs)
print("")
print("ファイルごとに各文字列の個数をカウントします")
COUNT_COLUMNS = ["col","empty", "space", "-", "―", "—", "--", "Na", "na", "N/A", "n/a", "None", "none", "NULL", "null", "0", "alphabet"]
FINAL_COLUMNS = COUNT_COLUMNS + ["ファイル名", "会計年"] # 追跡を明確にするため
df_placeholder_counts = pd.DataFrame(columns=FINAL_COLUMNS)
current_index = 0
for (filename,year), df in df_DATAs_BY_ALL_FILEs.items():
    for col in df:
        ser = df[col].fillna(pd.NA)
        ser_str = ser.astype(str)
        #display(ser_str)
        row_data_tuple = (
            col,
            (ser_str == '').sum(), (ser_str == ' ').sum(),
            (ser_str == '-').sum(), (ser_str == '―').sum(), (ser_str == '—').sum(), (ser_str == '--').sum(),
            (ser_str == 'Na').sum(), (ser_str == 'na').sum(), (ser_str == 'N/A').sum(), (ser_str == 'n/a').sum(),
            (ser_str == 'None').sum(), (ser_str == 'none').sum(), (ser_str == 'NULL').sum(), (ser_str == 'null').sum(),
            (ser_str == '0').sum(), 
            ser_str.str.contains('[A-Za-z]', na=False).sum()
        )
        df_placeholder_counts.loc[current_index, COUNT_COLUMNS] = row_data_tuple
        df_placeholder_counts.loc[current_index, "ファイル名"] = filename
        df_placeholder_counts.loc[current_index, "会計年"] = year

        current_index += 1
display(df_placeholder_counts)




In [ ]:
print("")
print("欠損値と疑われるコードを出力します。")
df = df_placeholder_counts.drop(["ファイル名","会計年"],axis=1)
df = df.groupby("col").sum().sum()
display(df)

print("")
print("疑われるコードが含まれている列を出力します")
df = df_placeholder_counts.drop(["ファイル名","会計年"],axis=1)
df = df.groupby("col").sum().sum()
pl_list = df[df >0].index.tolist()
df = df_placeholder_counts.drop(["ファイル名","会計年"],axis=1)
for value in pl_list:
    list = df[df[value]>0]["col"].unique()
    print("--------------",value,"のチェック----------------------------")
    display(list)

print("")
print("アルファベットが含まれているコードを出力します。")
for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
    df_tmp = df[df["コード"].str.contains("[A-Za-z]", na=False)]
    code_list = df_tmp["コード"].to_list()
    print(code_list)

print("")
print("ゼロが登録されているファイルと列を出力します。")
df = df_placeholder_counts
df_0 = df.groupby(["ファイル名","col"])["0"].sum()
df_0 = df_0[df_0>0].reset_index()
print(df_0)
print(df_0['col'].dtype)

print("")
print("4つのファイルごとに該当列のリストを準備します")
df_temp = df_0[['ファイル名', 'col']]
grouped = df_temp.groupby('ファイル名')
col_lists = {
    filename: group['col'].tolist()  # 各グループから 'col' 列を抽出し、tolist()でPythonリストに変換
    for filename, group in grouped
}
print(col_lists)

print("")
print("ゼロの含まれるコードのデータを確認します。")
print("まずは年でわかれた各ファイルをつなぎ合わせます")
df_merge = {}
for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
    if filename not in df_merge:
        df_merge[filename] = []
    df_merge[filename].append(df)
df_final = {}
for filename in df_merge:
    df_final[filename]  = pd.concat(df_merge[filename])
display(df_final["fy-balance-sheet.csv"])

print("")
print("ファイルごとにゼロをもつコードの値をチェックします。")
filename = "fy-profit-and-loss.csv"
print("ファイル名:",filename)
df_all = df_final[filename]
for col_value in col_lists[filename]:
    #print(filename, col)
    df = df_all[df_all[col_value] == "0"]
    code_list = df["コード"].unique()
    print("-----------------------",col_value,"-----------------------")
    for code in code_list:
        display(df_all[df_all["コード"] == code])

### データ型の整合性確認
ロードしたファイルの方が各列で共通になるようにします。
<details>
<summary><b>結果</b></summary>
以下のように処理しています。

- コード：string
- 年度：timestump
- その他：float
</details>

In [ ]:
print("")
print("ファイル毎にデータを取得します")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"]
)

print("")
print("ハイフンを欠損値にします。")
df_missing_value_change = {
    (filename, year): df.replace("-", pd.NA)
    for (filename, year), df in df_DATAs_BY_ALL_FILEs.items()
}
display(df_missing_value_change)

print("")
print("数値化できるものはfloat化します。")
exclude_cols = ["コード", "年度"]  # 数値変換したくない列
def safe_to_numeric(x):
    try:
        return pd.to_numeric(x)
    except Exception:
        return x  # 数値変換できなければそのまま返す
df_type_to_float = {
    key: df.assign(
        **{
            col: df[col].apply(safe_to_numeric) 
            for col in df.columns if col not in exclude_cols
        }
    )
    for key, df in df_missing_value_change.items()
}
display(df_type_to_float)


In [ ]:
print("")
print("各列のdtypeと要素のtypeを出力します。")
def check_column_types(df_DATAs_BY_ALL_FILEs):
    COUNT_COLUMNS = ["filename", "year", "col", "dtype", "sample_types"]
    df_type = pd.DataFrame(columns=COUNT_COLUMNS)
    current_index = 0
    for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
        for col in df.columns:
            # 実際のデータ型を確認
            dtype = df[col].dtype
            sample_types = df[col].dropna().map(type).unique()
            row_data_tuple = (filename, year, col, dtype, sample_types)
            df_type.loc[current_index, COUNT_COLUMNS] = row_data_tuple
            #print(row_data_tuple)
            current_index +=1
    return df_type
df_type = check_column_types(df_type_to_float)
display(df_type)

print("")
print("dtypeがobjectの要素を検索します。")
display(df_type[df_type["dtype"] == "object"]["col"].unique())

print("")
print("dtypeがobjectの要素をstrにします。")
df = df_type_to_float
df_type_change = {
    #(filename, year): df.astype({col: 'string' for col in df.select_dtypes(include='object').columns})
    (filename, year): (df.assign(コード=df["コード"].astype("string")) if "コード" in df.columns else df)
    for (filename, year), df in df_type_to_float.items()
}
#display(df_type_change)
print("")
print("日付はdatetimeにします。")
df_type_change = {
    (filename, year): df.assign(年度=pd.to_datetime(df["年度"], format="%Y/%m"))
    for (filename, year), df in df_type_change.items()
}
print("")
print("変更されているか確認します。")
df_type = check_column_types(df_type_change)
#display(df_type[df_type["dtype"] == "object"]["col"].unique())
display(df_type)

print("")
print("最後にsample_typesが正しいか確認します")
def chk_sample_type(df_type):
    SAMPLE_TYPES_COLUMNS = ["col", "sample_types"]
    df_sample_types = pd.DataFrame(columns=SAMPLE_TYPES_COLUMNS)
    col_list = df_type["col"].unique().tolist()
    current_index=0
    for col in col_list:
        row_data_tuple = (
            col,
            df_type[df_type["col"]==col]["sample_types"].astype(str).unique()
        )
        df_sample_types.loc[current_index, SAMPLE_TYPES_COLUMNS] = row_data_tuple
        current_index +=1
    return df_sample_types
df_sample_types = chk_sample_type(df_type)
#display(df_sample_types)

print("")
print("float以外が含まれている列をfloatだけにします")
float_cols = df_sample_types[df_sample_types["sample_types"].astype(str).str.contains("int")]
float_cols = float_cols["col"].to_list()
df_type_change = {
    (filename, year): df.assign(**{
        col: df[col].astype(float)
        for col in float_cols
        if col in df.columns
    })
    for (filename, year), df in df_type_change.items()
}
df_type = check_column_types(df_type_change)
df_sample_types = chk_sample_type(df_type)
display(df_sample_types)

### 初期品質の確認

指標となる以下の数を確認し、必要な処理のプロセスを決定します。

- コード
- 年度
- 列

<summary><b>結果</b></summary>

</details>

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_rows', 60)

print("")
print("処理済みデータをロードします。")
df_PROCESSED = df_type_change
#display(df_PROCESSED)

print("")
print("企業コード数、エンドの数、列数をファイルごとにまとめます。")
def get_file_info(df_PROCESSED):
    df_file_info = pd.DataFrame(columns=["file","year","code_counts","year_counts","column_counts"])
    for (filename, year), df in df_PROCESSED.items():
        # len(df_file_info)を次の新しい行のインデックスとして明示的に指定します。
        df_file_info.loc[len(df_file_info)] = [
            filename, year, df["コード"].nunique(), df["年度"].nunique(), df.columns.nunique()
        ]
    return df_file_info
df_file_info = get_file_info(df_PROCESSED)
#display(df_file_info)

print("")
print("各ファイルの年推移を可視化します。")
fig = px.line(df_file_info,x="year",y="code_counts",color="file")
#fig.show()
fig = px.line(df_file_info,x="year",y="year_counts",color="file")
#fig.show()
fig = px.line(df_file_info,x="year",y="column_counts",color="file")
#fig.show()

print("")
print("最新年のファイルに登録されている年度を表示します。")
df = df_PROCESSED[(settings["files"]["raw"][0],2025)]
#display(df["年度"].unique())

print("")
print("最新年に登録されている過去の年度と過去のデータが同じか確認します")
print("まずは、年度を限定して、コードの一覧を見ます")
#display(df[df["年度"]=="2024/12"]["コード"])

print("")
print("コードを限定し、過去のデータファイルと同じかどうか調べます。")
df = df_PROCESSED[(settings["files"]["raw"][0],2025)]
#display(df[df["コード"]=="130A"])
df = df_PROCESSED[(settings["files"]["raw"][0],2024)]
#display(df[df["コード"]=="130A"])

print("")
print("最新年度にある過去のデータを取得し、過去のデータを更新します")
def update_df(df_by_files, latest_year):
    df_all_datas_by_files = {}
    cutoff_date = cutoff_date = pd.to_datetime(f"{str(latest_year)}-01-01")
    #print(cutoff_date)
    all_files = {key[0] for key in df_by_files.keys()}
    for file in all_files:
        df_source_raw = df_by_files.get((file, latest_year))
        if df_source_raw is None:
            print(f"警告: {file} の2025年データが見つかりません。スキップします。")
            continue
        # フィルタリング条件を作成: '年度'が基準日より小さいデータのみ抽出
        df_update_source = df_source_raw[df_source_raw["年度"] < cutoff_date]
        df_update_source = df_update_source.set_index(["コード", "年度"])
        #display(df_update_source)
        # --- 2. 過去の全年のデータを更新 ---
        all_years = {key[1] for key in df_by_files.keys() if key[0] == file}
        for year in all_years:
            df_target = df_by_files.get((file, year))
            if df_target is None:
                continue
            df_target_indexed = df_target.set_index(["コード", "年度"])

            df_target_indexed.update(df_update_source)
            df_target_indexed = df_target_indexed.reset_index()
            df_all_datas_by_files[(file, year)] = df_target_indexed
    return df_all_datas_by_files

df_PROCESSED_AFTER = update_df(df_PROCESSED, 2025)

print("")
print("正しく処理が行われ、最新年に登録されている過去の年度と過去のデータが同じか確認します。")
df = df_PROCESSED_AFTER[(settings["files"]["raw"][0],2025)]
display(df[df["コード"]=="130A"])
df = df_PROCESSED_AFTER[(settings["files"]["raw"][0],2024)]
display(df[df["コード"]=="130A"])

print("")
print("最後に最新年度にある過去のデータを消去します。")
cutoff_date = pd.to_datetime("2025-01-01")
for file, year in df_PROCESSED_AFTER.keys():
    if year == 2025:
        df = df_PROCESSED_AFTER[(file,year)]
        df = df[df["年度"] >= cutoff_date]
        df_PROCESSED_AFTER[(file,year)] = df
print("")
print("各ファイルの年推移をもう一度可視化し、改善していることを確かめます。") 
df_file_info_after = get_file_info(df_PROCESSED_AFTER)
df_file_info_after["区分"] = "処理後"
df_file_info_after = df_file_info_after.set_index(["file", "year"]).sort_index()

df_file_info = get_file_info(df_PROCESSED)
df_file_info["区分"] = "処理前"
df_file_info = df_file_info.set_index(["file", "year"]).sort_index()
df_compare = pd.concat([df_file_info,df_file_info_after]).reset_index()
display(df_compare)
fig = px.line(df_compare, x="year", y="code_counts", color="file", line_dash="区分", markers=True)
fig.update_layout(title="処理前後のコード数推移", xaxis_title="年度", yaxis_title="コード数")
fig.show()

### 上場企業データのクリーニング

<summary><b>結果</b></summary>

</details>

In [15]:
print("\nロードします。")
df_code_info = dl.load_on_startup(settings["data_path"]["reference"], "", settings["files"]["reference"])
#display(df_code_info)
dl.chk_file_missing(df_code_info)

print("\n欠損値表現を確認します。")
df_missing_values_expression = cu.chk_missing_values_expression(
    df_code_info, filename=settings["files"]["reference"], option_value=""
)
#display(df_missing_values_expression)

print("\n欠損値処理をします。")
df_code_info_missing_change = df_code_info.replace({"-": pd.NA, "": pd.NA})

display(df_code_info_missing_change)
display(df_code_info_missing_change["33業種コード"].map(type).unique())

print("\n数値列を Int64 に変換します。")
numeric_cols = ["17業種コード", "規模コード"]
df = df_code_info_missing_change
df = cu.convert_columns_type(df, numeric_cols, "int", True)

print("\nstring 型に統一します。")
for col in df.columns:
    if col not in numeric_cols:
        df[col] = df[col].astype("string")

print("\n日付列を datetime に変換します。")
df["日付"] = pd.to_datetime(df["日付"], errors="coerce", format="%Y%m%d")
df_type_change = df

print("")
print("欠損値表現を確認します。")
df_missing_values_expression = cu.chk_missing_values_expression(df_type_change, filename=settings["files"]["reference"], option_value="")
#display(df_missing_values_expression)

print("")
print("データ型を確認します。")
df_type = cu.chk_dtype(df_type_change, filename=settings["files"]["reference"], option_value="")
#display(df_type)


ロードします。
✅ ファイルは欠損していません。

欠損値表現を確認します。

欠損値処理をします。


,日付,コード,銘柄名,市場・商品区分,33業種コード,33業種区分,17業種コード,17業種区分,規模コード,規模区分
0,20250930,1301,極洋,プライム（内国株式）,50,水産・農林業,1,食品,7,TOPIX Small 2
1,20250930,1305,ｉＦｒｅｅＥＴＦ ＴＯＰＩＸ（年１回決算型）,ETF・ETN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,20250930,1306,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF・ETN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,20250930,1308,上場インデックスファンドＴＯＰＩＸ,ETF・ETN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,20250930,1309,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF・ETN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...
4398,20250930,9991,ジェコス,プライム（内国株式）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
4399,20250930,9993,ヤマザワ,スタンダード（内国株式）,6100,小売業,14,小売,<NA>,<NA>
4400,20250930,9994,やまや,スタンダード（内国株式）,6100,小売業,14,小売,<NA>,<NA>
4401,20250930,9996,サトー商会,スタンダード（内国株式）,6050,卸売業,13,商社・卸売,<NA>,<NA>


array([<class 'str'>, <class 'pandas._libs.missing.NAType'>], dtype=object)


数値列を Int64 に変換します。

列 '17業種コード' を int に変換中...

列 '規模コード' を int に変換中...

string 型に統一します。

日付列を datetime に変換します。

欠損値表現を確認します。

データ型を確認します。
